In [208]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
using LinearAlgebra
import ForwardDiff
using Random

  Activating environment at `~/homework/optimal_control_and_rl/CMU-16745-Project/Project.toml`


In [209]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","quadrotor.jl"))

animate_quadrotor (generic function with 1 method)

In [236]:
# x, y, z, radius
function create_obstacles()
    return [[1.0,-1.0,-0.5,0.5], [-1.0,-1.0,-0.5,0.5],[0.0,-2.5,-0.5,0.5]]#[[2.5, -2.5, -1, 1]]#, [-2.5, -2.5, -1, 1], [0, -2.5, -1, 1]]
end

create_obstacles (generic function with 1 method)

In [238]:
# problem size 
nx = 16
nu = 4
dt = 0.1
tf = 12.0
t_vec = 0:dt:tf 
N = length(t_vec)

# LQR cost 
Q = diagm(ones(nx))
R = 0.1*diagm(ones(nu))
Qf = 10*diagm(ones(nx))

# setup indexing 
idx = create_idx(nx,nu,N)

# initial and goal states 
x_goals = zeros(N,nx)

xic = zeros(nx)
xic[3] = -1.5

x_mid = zeros(nx)
x_mid[1] = 2.5
x_mid[2] = -2.5
x_mid[3] = -1.5

x_end = zeros(nx)
x_end[1] = -2.5
x_end[2] = -2.5
x_end[3] = -1.5

# Setup goal times
for i = 1:N÷3
    x_goals[i,:] = x_mid
end
for i = N÷3:2*N÷3
    x_goals[i,:] = x_end
end
for i = 2*N÷3:N
    x_goals[i,:] = xic
end

# Setup Jump map points
traj_length = N÷3
M1 = vcat([ (i-1)*10      .+ (1:traj_length)   for i = 1:1]...) # stack the set into a vector
M2 = vcat([((i-1)*10 + traj_length) .+ (1:traj_length)   for i = 1:1]...) # stack the set into a vector
M3 = vcat([((i-1)*10 + 2*traj_length) .+ (1:traj_length)   for i = 1:1]...) # stack the set into a vector
        
# load all useful things into params 
model_w_payload = (mass_quad=0.5,
        mass_payload=0.2,
        ℓ = 0.1750, # Quad radius
        L_p = 1, # Pendulum length
        J=Diagonal([0.0023, 0.0023, 0.004]),
        gravity=[0;0;9.81],
        kf=1.0,
        km=0.0245, dt = dt)
model_wo_payload = (mass_quad=0.5,
        mass_payload=0.05,
        ℓ = 0.1750, # Quad radius
        L_p = 1, # Pendulum length
        J=Diagonal([0.0023, 0.0023, 0.004]),
        gravity=[0;0;9.81],
        kf=1.0,
        km=0.0245, dt = dt)    

params = (
        Q = Q, 
        R = R, 
        Qf = Qf, 
        xic = xic,
        x_goals = x_goals,
        dt = dt, 
        N = N, 
        idx = idx, 
        model_w_payload=model_w_payload,
        model_wo_payload=model_wo_payload,
        M1=M1,
        M2=M2,
        M3=M3,
        obstacles=create_obstacles(),
        obstacle_buffer=params.model_w_payload.L_p
)

    
X, U, t_vec = solve_quad(params, verbose=true);

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:  4730528
Number of nonzeros in inequality constraint Jacobian.:  1756432
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     2416
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      240
                     variables with only upper bounds:      120
Total number of equality constraints.................:     1958
Total number of inequality constraints.......

In [284]:
import MeshCat as mc
using CoordinateTransformations, Rotations, Colors
using GeometryBasics
import RobotDynamics as RD
using StaticArrays

function set_mesh!(vis, pole_length, obstacles)
    
    obj = joinpath(@__DIR__, "utils/quadrotor.obj")
    
    robot_obj = mc.MeshFileGeometry(obj)
    quad_mat = mc.MeshPhongMaterial(color=colorant"black")
    
    pole_mat = mc.MeshPhongMaterial(color=colorant"blue")
    mass_mat = mc.MeshPhongMaterial(color=colorant"red")
    pole = mc.Cylinder(mc.Point3f0(0,0,0),Point3f0(0,0,-pole_length),0.01f0)
    mass = mc.HyperSphere(mc.Point3f0(0,0,0), 0.15f0)
    
    obs_mat = mc.MeshPhongMaterial(color=colorant"green")
    for i = 1:length(obstacles)
        obs_edu = [obstacles[i][2]; obstacles[i][1]; -obstacles[i][3]]
        obs = mc.HyperSphere(mc.Point3f0(obs_edu[1], obs_edu[2], obs_edu[3]), Float32(0.33))
        mc.setobject!(vis["obs" * string(i)], obs, obs_mat)
    end
#     obs = mc.HyperSphere(mc.Point3f0(-0.5,-0.5,-0.5), 0.5f0)
    
    mc.setobject!(vis["quad"], robot_obj, quad_mat)
    mc.setobject!(vis["pole"], pole, pole_mat)
    mc.setobject!(vis["mass"], mass, mass_mat)
end
#Quaternion stuff
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I]
function qtoQ(q)
    return H'*T*L(q)*T*L(q)*H
end
function G(q)
    G = L(q)*H
end
function rptoq(ϕ)
    (1/sqrt(1+ϕ'*ϕ))*[1; ϕ]
end
function qtorp(q)
    q[2:4]/q[1]
end

function visualize!(vis, x::StaticVector, attached, last_pos, params)
    quad_rot_mat = dcm_from_mrp(x[4:6])

    # NED = North, East, Down (coord frame used in paper)
    # ENU = East, North, Up (coord frame used by meshcat)
    quad_ned = x[1:3]
    quad_enu = [quad_ned[2]; quad_ned[1]; -quad_ned[3]]
    
    mass_ned = [x[7] + quad_ned[1]; x[8] + quad_ned[2]; sqrt(params.model_w_payload.L_p^2 - x[7:8]' * x[7:8]) + quad_ned[3]]
    mass_enu = [mass_ned[2]; mass_ned[1]; -mass_ned[3]]
    
    # Find rotation that points a vector from quad to mass
    up = [0;0;1]
    dir = quad_enu - mass_enu
    q = [0; cross(up, dir)]
    q[1] = sqrt(norm(up)^2 * norm(dir)^2) + dot(up, dir)
    q = normalize(q)
    pole_rot_mat = qtoQ(q)
    
    mc.settransform!(vis["quad"], compose(Translation(quad_enu...), LinearMap(quad_rot_mat)))
    mc.settransform!(vis["pole"], compose(Translation(quad_enu...), LinearMap(pole_rot_mat)))
    
    
    if (attached)
        mc.settransform!(vis["mass"], compose(Translation(mass_enu...)))
        return mass_enu
    else
        mc.settransform!(vis["mass"], compose(Translation(last_pos...)))
        return last_pos
    end
    
end

function visualize!(vis, tf::Real, X, params)
    fps = Int(round((length(X)-1)/tf))
    anim = mc.Animation(fps)
    x_dim = size(X[1])[1]
    last_pos = params.x_goals[params.N÷2,:][1:3]
    last_pos[3] += params.model_w_payload.L_p
    last_pos = [last_pos[2]; -last_pos[1]; -last_pos[3]]
    
    
    offset = 0
    for (k,x) in enumerate(X)
        mc.atframe(anim, k) do
            x = X[k]
            
            if (k > 40) && (k < 80)
                if offset < 1
                    offset += 1
                else
                    last_pos = visualize!(vis, SVector{x_dim}(x), true, last_pos, params) 
                end
            else
                visualize!(vis, SVector{x_dim}(x), false, last_pos, params) 
            end
        end
    end
    mc.setanimation!(vis, anim)
end

visualize! (generic function with 2 methods)

In [267]:
vis = mc.Visualizer()
mc.render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8758


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8758))

In [285]:
set_mesh!(vis, params.model_w_payload.L_p, params.obstacles)

x_dim = size(X[1])[1]
X1 = [SVector{x_dim}(x) for x in X];
visualize!(vis, t_vec[end], X1, params)